In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('Новосибирск-5.xls', index_col=0, skiprows=1, usecols='A:I')
df.head()


,Севооборот,минус 3 года,минус 2 года,минус 1 год,культура,год,урожай,примечание
№,,,,,,,,
1,0,пшеница,пшеница,пшеница,пшеница,1999,11.1,предшественники фиктивные
2,0,пшеница,пшеница,пшеница,пшеница,2000,31.7,бессменная пшеница
3,0,пшеница,пшеница,пшеница,пшеница,2001,25.3,NaN
4,0,пшеница,пшеница,пшеница,пшеница,2002,13.6,NaN
5,0,пшеница,пшеница,пшеница,пшеница,2003,8.2,NaN


## Промежуточная матрица с известными значениями

In [3]:
def find_avg(df, forbear, crop):
    if crop == 'пар':
        return 0, 'получено экспереметально'
    
    # Фильтрация строк по значениям в столбцах 'минус 1 год' и 'культура'
    filtered_df = df[(df['минус 1 год'] == forbear) & (df['культура'] == crop)]
    
    # Если есть совпадения, возвращаем значение из столбца 'avg'
    if not filtered_df.empty:
        return filtered_df['урожай'].mean(), 'получено экспереметально'
    return None, None
    


crops = {'предшественник':[], 'культура':[], 'avg':[], 'примечание':[]}
unique_crop = df['культура'].unique()
for forbear in unique_crop:
    for crop in unique_crop:
        crops['предшественник'].append(forbear)
        crops['культура'].append(crop)
        avg, remark = find_avg(df, forbear, crop)
        crops['avg'].append(avg)
        crops['примечание'].append(remark)
tensor = pd.DataFrame(crops)
pd.set_option('display.max_rows', None)
tensor


,предшественник,культура,avg,примечание
0,пшеница,пшеница,17.599048,получено экспереметально
1,пшеница,овес,18.985714,получено экспереметально
2,пшеница,ячмень,20.750000,получено экспереметально
3,пшеница,пар,0.000000,получено экспереметально
4,пшеница,оз. рожь,NaN,None
5,пшеница,донник(вика)клевер,NaN,None
6,пшеница,вика-овес,NaN,None
7,пшеница,рапс,NaN,None
8,пшеница,клевер,NaN,None
9,пшеница,вика-овес (з/м),NaN,None


## Заполняем пустые ячейки, умножая среднюю урожайность культуры на среднее влияние, которое оказывает предшественник

In [6]:
import numpy as np

def calculate_avg(df, forbear, crop):
    df_forbear = df[(df['минус 1 год'] == forbear) & (df['культура'] != 'пар')]
    unique_crop = df_forbear['культура'].unique()
    if unique_crop.size == 0:
        return None, None
    odds = [df_forbear[df_forbear['культура'] == crop]['урожай'].mean() 
            / df[df['культура'] == crop]['урожай'].mean() for crop in unique_crop]
    return sum(odds) / len(odds) * df[df['культура'] == crop]['урожай'].mean(), unique_crop


for index, row in tensor.iterrows():
    forbear, crop, avg = row['предшественник'], row['культура'], row['avg']
    
    if pd.isna(avg):
        calculate_avg_result, source = calculate_avg(df, forbear, crop)
        if calculate_avg_result is not None: 
            tensor.at[index, 'avg'] = calculate_avg_result
            tensor.at[index, 'примечание'] = f'получено из {", ".join(source)}'

     
tensor


,предшественник,культура,avg,примечание
0,пшеница,пшеница,17.599048,получено экспереметально
1,пшеница,овес,18.985714,получено экспереметально
2,пшеница,ячмень,20.750000,получено экспереметально
3,пшеница,пар,0.000000,получено экспереметально
4,пшеница,оз. рожь,27.769808,"получено из пшеница, овес, ячмень"
5,пшеница,донник(вика)клевер,30.959627,"получено из пшеница, овес, ячмень"
6,пшеница,вика-овес,23.592412,"получено из пшеница, овес, ячмень"
7,пшеница,рапс,5.908361,"получено из пшеница, овес, ячмень"
8,пшеница,клевер,39.717313,"получено из пшеница, овес, ячмень"
9,пшеница,вика-овес (з/м),26.973991,"получено из пшеница, овес, ячмень"


In [7]:
tensor.to_excel('tensor.xlsx')